# Homework 3
#### Team: Jair Aguirre and William Gottschalk, Friday Section

In [6]:
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
imp = Imputer(missing_values='NaN', strategy='median', axis=0)
lb = LabelBinarizer()
X = imp.fit_transform(pd.read_csv('trainingData.txt', sep='\t', header=None))
from collections import Counter

Y = pd.read_csv('trainingTruth.txt', sep='\t', header=None)
sm = SMOTE(kind='regular')
X_resampled, Y_resampled = sm.fit_sample(X, Y)
r, _ = Y.values.shape

print(
    Counter(Y_resampled), 
    Counter(Y.values.reshape(r, ))
)

Y_resampled = lb.fit_transform(Y_resampled)
print(Y_resampled)

/Users/willg/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Counter({1: 5667, 4: 5667, 3: 5217, 2: 3893}) Counter({1: 5667, 3: 5217, 2: 3893, 4: 2601})
[[0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 ..., 
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    StandardScaler().fit_transform(X_resampled) , Y_resampled, test_size=0.5, random_state=0
)

print(X_train.shape, y_train.shape)
print(X_train[:3])
print(y_train[:3])

(10222, 334) (10222, 4)
[[-2.69645963 -0.50278739 -1.02432391 ...,  1.24144138  1.37329155
  -1.36227482]
 [ 0.31885796  0.92483342  1.47501942 ..., -0.34250057  0.41603692
   2.49825508]
 [ 0.64052788  0.31117727  0.05487849 ...,  2.2434861   0.8015509
  -0.60216839]]
[[0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]]


In [9]:
# clf = RandomForestClassifier(n_estimators=30)
# scores = cross_val_score(clf, X_train, y_train, scoring='f1', n_jobs=-1)

In [17]:
parameters = {
    "estimator__n_estimators": [100],
    "estimator__min_samples_split": [2, 10, 30]
}

tree = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1))
clf = GridSearchCV(tree, parameters, cv=10)

clf.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
          n_jobs=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'estimator__min_samples_split': [2, 10, 30], 'estimator__n_estimators': [100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [18]:
# columns = pd.DataFrame(X).columns
# sorted_features = sorted(zip(columns, clf.estimator.estimator.feature_importances_), key = lambda x: x[1], reverse=True)
# print(sorted_features[:27])

# fig_size = plt.rcParams["figure.figsize"]
# fig_size = (24, 12)

# plt.hist(clf.estimator.estimator.feature_importances_, bins=20, align='right', orientation='horizontal', rwidth=5)

print(clf.score(X_test, y_test))
print(clf.best_params_)
print(clf.best_score_)

0.223048327138
{'estimator__min_samples_split': 2, 'estimator__n_estimators': 100}
0.209352377226


In [163]:
trees = OneVsRestClassifier(RandomForestClassifier(n_estimators=100))
trees.fit(X_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
          n_jobs=1)

In [168]:
# outcomes = pd.DataFrame({"predicts" : tree.predict(X_test)})
# outcomes["actuals"] = y_test
# outcomes["diffs"] = outcomes.predicts - outcomes.actuals == 0

# num, _ = outcomes.shape
# x = Counter(outcomes.diffs.values)
# x[False] = x[False] / num
# x[True] = x[True] / num
# x
best_features = []
for tree in trees.estimators_:
    sorted_features = sorted(
        zip(columns, tree.feature_importances_),
        key = lambda x: x[1], reverse=True)
    best_features.append(sorted_features[:10])
    array = []
    for i, n in sorted_features[:25]:
        array.append(i)
    print(array)
pd.DataFrame(best_features).T

[114, 47, 325, 243, 215, 23, 303, 42, 163, 157, 279, 78, 87, 259, 146, 84, 91, 268, 194, 29, 139, 202, 298, 199, 244]
[222, 197, 209, 125, 168, 28, 218, 167, 27, 89, 279, 249, 80, 206, 11, 170, 109, 61, 224, 73, 154, 144, 323, 232, 69]
[78, 303, 325, 264, 108, 114, 153, 42, 142, 47, 216, 146, 1, 213, 163, 298, 15, 229, 294, 274, 98, 241, 244, 127, 305]
[23, 167, 24, 243, 108, 333, 69, 6, 279, 212, 8, 162, 168, 197, 222, 247, 74, 117, 224, 96, 299, 119, 28, 211, 249]


,0,1,2,3
0,"(114, 0.015966224112)","(222, 0.0128230447711)","(78, 0.0204454124396)","(23, 0.0120089165076)"
1,"(47, 0.0147081268511)","(197, 0.0122509892884)","(303, 0.0166003666736)","(167, 0.0103367795134)"
2,"(325, 0.0138667946906)","(209, 0.0100858811745)","(325, 0.0136341644823)","(24, 0.00764497409063)"
3,"(243, 0.0131515883515)","(125, 0.00977806328079)","(264, 0.0129327987654)","(243, 0.00758605781431)"
4,"(215, 0.0118347302151)","(168, 0.00922426977286)","(108, 0.0127265094412)","(108, 0.00701609632686)"
5,"(23, 0.0117863184286)","(28, 0.0079681929361)","(114, 0.0122006842831)","(333, 0.00700287301022)"
6,"(303, 0.0114860504419)","(218, 0.00792920037299)","(153, 0.0117281504015)","(69, 0.00690090772046)"
7,"(42, 0.0114695511235)","(167, 0.00702455741537)","(42, 0.0108859155115)","(6, 0.0064759173909)"
8,"(163, 0.0103101584103)","(27, 0.00671988261403)","(142, 0.00884147216672)","(279, 0.00616803768163)"
9,"(157, 0.010301732716)","(89, 0.00668069215395)","(47, 0.00869648458595)","(212, 0.00609326444668)"


In [109]:
[(114, 0.016773333754614105), (47, 0.016691345349158802), (325, 0.016282379470957151), (243, 0.014373052579774247), (215, 0.013169360497468426), (303, 0.012283564058043121), (87, 0.010713647845129768), (163, 0.010625715826252363), (194, 0.010437116573125238), (42, 0.010436283628632682), (23, 0.0096253217870516557), (298, 0.0095835788142584408), (78, 0.0091603913303178047), (29, 0.0090684529057932367), (146, 0.0088781843483942708), (259, 0.0086795439530645748), (244, 0.0085123624104761347), (91, 0.0084532449751195879), (268, 0.0081362892224597031), (45, 0.0080564564860679186), (279, 0.0079504227941750057), (202, 0.0078419923051651873), (157, 0.0078049684742194491), (174, 0.0077461212057034837), (84, 0.0074631488463187748), (139, 0.0071864555900003849), (15, 0.0070088788618062683)]

[(114, 0.016773333754614105),
 (47, 0.016691345349158802),
 (325, 0.01628237947095715),
 (243, 0.014373052579774247),
 (215, 0.013169360497468426),
 (303, 0.012283564058043121),
 (87, 0.010713647845129768),
 (163, 0.010625715826252363),
 (194, 0.010437116573125238),
 (42, 0.010436283628632682),
 (23, 0.009625321787051656),
 (298, 0.00958357881425844),
 (78, 0.009160391330317805),
 (29, 0.009068452905793237),
 (146, 0.00887818434839427),
 (259, 0.008679543953064575),
 (244, 0.008512362410476135),
 (91, 0.008453244975119588),
 (268, 0.008136289222459703),
 (45, 0.008056456486067919),
 (279, 0.007950422794175006),
 (202, 0.007841992305165187),
 (157, 0.007804968474219449),
 (174, 0.007746121205703484),
 (84, 0.007463148846318775),
 (139, 0.007186455590000385),
 (15, 0.007008878861806268)]